<a href="https://colab.research.google.com/github/tsaniarisqi/Machine_Learning_2022/blob/main/js03/js03_Tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Nama:  Tsania Risqi El Istiqomah
*   Kelas: TI-4J
*   NIM: 2241727041

# Tugas

## Tugas 1

Buatlah model klasfikasi Multinomial Naive Bayes dengan ketentuan,

1. Menggunakan data `spam.csv`
2. Fitur `CountVectorizer` dengan mengaktifkan **stop_words**
3. Evaluasi hasilnya

## Tugas 2

Buatlah model klasfikasi Multinomial Naive Bayes dengan ketentuan,

1. Menggunakan data `spam.csv`
2. Fitur `TF-IDF` dengan mengaktifkan **stop_words**
3. Evaluasi hasilnya dan bandingkan dengan hasil tugas 1.
4. Berikan kesimpulan fitur mana yang terbaik pada kasus data `spam.csv`

# Tugas 1

## Load Data

In [17]:
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/dataset/js03/spam.csv', encoding='latin-1') # spesifiksi encoding diperlukan karena data tidak menggunakan UTF-8

df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Terdapat 3 kolom yang tidak bermanfaat untuk proses selanjutnya, maka kita perlu membuang kolom tersebut. Selain itu, untuk memudahkan pembacaan data, kita juga akan mengubah nama kolom **v1** yang berupa label dan **v2** yang berupa teks sms

## Pra Pengolahan Data
Beberapa hal yang akan dilakukan pada tahap ini yaitu,

1. Drop kolom yang tidak digunakan
2. Ubah nama kolom v1 (label) dan v2 (teks sms)
3. Inspeksi Data
4. Encode label
5. Memisahkan fitur dengan label

### Drop Kolom

In [46]:
# Drop 3 kolom terakhir dengan fungsi iloc
df = df.drop(df.iloc[:,2:], axis=1)

# Cek data
df.head()

,Labels,SMS
0,NaN,"Go until jurong point, crazy.. Available only ..."
1,NaN,Ok lar... Joking wif u oni...
2,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,NaN,U dun say so early hor... U c already then say...
4,NaN,"Nah I don't think he goes to usf, he lives aro..."


### Ubah Nama Kolom V1 dan V2

In [47]:
# Data untuk rename kolom v1 dan v2
new_cols = {
    'v1': 'Labels',
    'v2': 'SMS'
}

# Rename nama kolom v1 dan v2
df = df.rename(columns=new_cols)

# cek data
df.head()

,Labels,SMS
0,NaN,"Go until jurong point, crazy.. Available only ..."
1,NaN,Ok lar... Joking wif u oni...
2,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,NaN,U dun say so early hor... U c already then say...
4,NaN,"Nah I don't think he goes to usf, he lives aro..."


### Inspeksi Data

In [48]:
# Cek Jumlah Data Per Kelas
print(df['Labels'].value_counts())
print('\n')

# Cek Kelengkapan Data
print(df.info())
print('\n')

# Cek Statistik Deskriptif
print(df.describe())

Series([], Name: Labels, dtype: int64)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Labels  0 non-null      float64
 1   SMS     5572 non-null   object 
dtypes: float64(1), object(1)
memory usage: 87.2+ KB
None


       Labels
count     0.0
mean      NaN
std       NaN
min       NaN
25%       NaN
50%       NaN
75%       NaN
max       NaN


### Encode Label

In [49]:
# Data untuk label
new_labels = {
    'spam': 1,
    'ham': 0
}

# Encode label
df['Labels'] = df['Labels'].map(new_labels)

# Cek data
df.head()

,Labels,SMS
0,NaN,"Go until jurong point, crazy.. Available only ..."
1,NaN,Ok lar... Joking wif u oni...
2,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,NaN,U dun say so early hor... U c already then say...
4,NaN,"Nah I don't think he goes to usf, he lives aro..."


### Memisahkan Fitur dengan Label

In [50]:
X = df['SMS'].values
y = df['Labels'].values

### Ekstrasi Fitur

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split data training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow = CountVectorizer(stop_words=["the","an","a","on","in","at","do","will"])


# Fitting dan transform X_train dengan CountVectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
# Mengapa hanya transform? Alasan yang sama dengan kasus pada percobaan ke-3
# Kita tidak menginginkan model mengetahui paramter yang digunakan oleh CountVectorizer untuk fitting data X_train
# Sehingga, data testing dapat tetap menjadi data yang asing bagi model nantinya
X_test = bow.transform(X_test)

## Training dan Evaluasi Model

In [37]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Inisiasi MultinomialNB
mnb = MultinomialNB()

# Fit model
mnb.fit(X_train, y_train)

# Prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# Evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train)

# Prediksi dengan data training
y_pred_test = mnb.predict(X_test)

# Evaluasi akurasi data training
acc_test = accuracy_score(y_test, y_pred_test)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.994166479694862
Hasil akurasi data test: 0.97847533632287


## Evaluasi Hasil 
Hasil dari mengaktifkan stop_words dengan fitur CountVectorizer dengan mengaktifkan stop_words sebagai berikut:
* Hasil akurasi data train: 0.994166479694862
* Hasil akurasi data test: 0.97847533632287

Hasil tersebut tidak jauh berbeda dengan hasil dari tidak mengaktifkan stop_words

# Tugas 2

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split data training dan testing
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=50)

# Inisiasi CountVectorizer
bow2 = TfidfVectorizer(stop_words=["the","an","a","on","in","at","do","will"])


# Fitting dan transform X_train dengan CountVectorizer
X_train2 = bow2.fit_transform(X_train2)

# Transform X_test
# Mengapa hanya transform? Alasan yang sama dengan kasus pada percobaan ke-3
# Kita tidak menginginkan model mengetahui paramter yang digunakan oleh CountVectorizer untuk fitting data X_train
# Sehingga, data testing dapat tetap menjadi data yang asing bagi model nantinya
X_test2 = bow2.transform(X_test2)

In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Inisiasi MultinomialNB
mnb2 = MultinomialNB()

# Fit model
mnb2.fit(X_train2, y_train2)

# Prediksi dengan data training
y_pred_train2 = mnb2.predict(X_train2)

# Evaluasi akurasi data training
acc_train2 = accuracy_score(y_train2, y_pred_train2)

# Prediksi dengan data training
y_pred_test2 = mnb2.predict(X_test2)

# Evaluasi akurasi data training
acc_test2 = accuracy_score(y_test2, y_pred_test2)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train2}')
print(f'Hasil akurasi data test: {acc_test2}')

Hasil akurasi data train: 0.9755440879515369
Hasil akurasi data test: 0.9461883408071748


## Evaluasi Hasil
Hasil dari mengaktifkan stop_words dengan fitur CountVectorizer dengan mengaktifkan stop_words sebagai berikut:
* Hasil akurasi data train: 0.9755440879515369
* Hasil akurasi data test: 0.9461883408071748

Hasil akurasi data train dan data test pada tugas 2 lebih menurun dari hasil akurasi data train dan data test pada tugas 1

## Kesimpulan
Dari evaluasi hasil diatas, maka dapat ditarik kesimpulan bahwa fitur terbaik pada kasus data spam.csv yaitu dengan menggunakan fitur CountVectorizer